In [ ]:
#Imports
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Initialize model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Load dataset
df = pd.read_csv('sample_data/QA_Dataset.csv')

In [ ]:
def generate_answer(question, context):
    input_text = f"Question: {question} Context: {context}"

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt")

    # Generate a response
    outputs = model.generate(inputs['input_ids'], max_length=100)

    # Decode and return the generated response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Function to find the most relevant context from the dataset
def find_relevant_context(question, df):
    # Use TF-IDF to measure similarity between the question and the dataset contexts
    vectorizer = TfidfVectorizer()

    # Combine the question with the dataset's contexts
    all_texts = df['Context'].tolist() + [question]

    # Create TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    # Compute cosine similarity between the question and each context
    similarities = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])

    # Find the index of the most similar context
    best_match_idx = similarities.argmax()

    # Return the best matching context
    return df.iloc[best_match_idx]['Context']

In [ ]:
# Function to answer a question based on the dataset
def answer_question(question, df):
    # Find the most relevant context from the dataset
    relevant_context = find_relevant_context(question, df)

    # Generate an answer using the found context
    answer = generate_answer(question, relevant_context)

    return answer

In [ ]:
# Example usage: Ask a question
user_question = "Why is the earth round?"
answer = answer_question(user_question, df)
print(f"Question: {user_question}")
print(f"Answer: {answer}")

Question: Why is the earth round?
Answer: housing bubble
